<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/LEE/GraphCodeBERT_1_N_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
!pip install datasets

In [40]:
import pandas as pd

In [41]:
#df = pd.read_csv('/content/drive/MyDrive/Dataset/prototype5.csv')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model train/final_train.csv')

In [ ]:
df

In [ ]:
df.columns

In [42]:
df = df[['CODE', 'IDX']]

In [ ]:
df.columns

In [ ]:
df

,CODE,IDX
0,void fz_init_cached_color_converter(fz_context...,1
1,"add_range(fz_context *ctx, pdf_cmap *cmap, uns...",10
2,"begin_softmask(fz_context *ctx, pdf_run_proces...",10
3,"add_range(fz_context *ctx, pdf_cmap *cmap, uns...",10
4,"pdf_show_image(fz_context *ctx, pdf_run_proce...",1
...,...,...
24717,int gs_main_init2aux(gs_main_instance * minst)...,0
24718,"pdf_show_char(fz_context *ctx, pdf_run_process...",0
24719,void goodG2BSink(wchar_t * * data);;\nstatic v...,0
24720,"ensure_solid_xref(fz_context *ctx, pdf_documen...",0


In [43]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [57]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels = 11)
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [58]:
import re

def preprocess(df, file_name):
    # 멀티 라인 주석 제거
    preprocess_df = df.replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    preprocess_df = preprocess_df.replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include <.*?>\n'), "", regex=True)
    # double quotes를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include \".*?\"\n'), "", regex=True)
    # 매크로 정의 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    preprocess_df = preprocess_df.replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    preprocess_df = preprocess_df.replace(re.compile(r'\n\s*\n'), "\n", regex=True)

    preprocess_df.to_csv(file_name, index=False)

def tokenized(examples):
  return tokenizer(examples['CODE'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [59]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='IDX',new_column_name='labels')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/24242 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [60]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=100)

In [61]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [62]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [63]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics


In [64]:
training_args = TrainingArguments(
    output_dir='./results',             # 학습 결과를 저장할 디렉터리
    num_train_epochs=7,                 # 데이터셋 크기에 맞춰 조정된 학습 에폭 수
    per_device_train_batch_size=16,     # 훈련 배치 크기, GPU 메모리에 맞게 조정 가능
    per_device_eval_batch_size=64,      # 평가 배치 크기, GPU 메모리에 맞게 조정 가능
    warmup_steps=500,                   # 워밍업 스텝 수, 총 스텝의 10%를 고려하여 조정 가능
    weight_decay=0.01,                  # 가중치 감소
    evaluation_strategy="epoch",        # 에폭마다 평가 수행
    logging_dir='./logs',               # 로그를 저장할 디렉터리
)

trainer = Trainer(
    model=model,                         # 학습시킬 모델
    args=training_args,                  # 학습 인자
    train_dataset=encoded_dataset['train'],         # 훈련 데이터셋
    eval_dataset=encoded_dataset['test'],           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 메트릭 계산 함수
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [65]:
gc.collect()
torch.cuda.empty_cache()

In [66]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9
1,0.675900,0.542773,0.816444,0.816444,0.821261,0.784502,0.816444,0.922890,0.550063,0.525843,1.000000,0.992278,0.992791,1.000000,0.540984,0.879121,0.621387
2,0.419300,0.382789,0.871442,0.871442,0.876054,0.850353,0.871442,0.945136,0.732299,0.620594,1.000000,0.991271,0.991786,1.000000,0.706827,0.890983,0.717241
3,0.250000,0.296086,0.911453,0.911453,0.911820,0.897790,0.911453,0.950467,0.816236,0.741294,1.000000,0.992263,0.992806,1.000000,0.810680,0.930114,0.790698
4,0.141400,0.329079,0.915578,0.915578,0.913537,0.902647,0.915578,0.950078,0.801712,0.728617,1.000000,0.993224,0.992806,1.000000,0.827878,0.937028,0.817294
5,0.114200,0.349829,0.934002,0.934002,0.934022,0.924999,0.934002,0.958566,0.871873,0.814882,0.999080,0.988281,0.988810,1.000000,0.867314,0.954151,0.820446
6,0.062000,0.335234,0.939777,0.939777,0.939615,0.932021,0.939777,0.962201,0.881481,0.809566,0.999080,0.989331,0.989754,1.000000,0.893855,0.961586,0.851182
7,0.042500,0.339046,0.941702,0.941702,0.941688,0.933601,0.941702,0.964804,0.887892,0.817621,0.999080,0.990291,0.990788,1.000000,0.886937,0.960298,0.858322


TrainOutput(global_step=7427, training_loss=0.2888620996019377, metrics={'train_runtime': 4029.9435, 'train_samples_per_second': 29.475, 'train_steps_per_second': 1.843, 'total_flos': 3.1255645971250176e+16, 'train_loss': 0.2888620996019377, 'epoch': 7.0})